In [1]:
from imutils import paths
import numpy as np
import imutils
import cv2

def find_marker(image):
    # преобразование изображения в оттенки серого, размытие и определение краев
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)
    # находим контуры на изображении и оставляем самый крупный;
    # в нашем случае это лист A4
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key = cv2.contourArea)
    # вычисляем ограничивающую рамку области листа А4 и возвращаем ее
    return cv2.minAreaRect(c)

In [2]:
def distance_to_camera(knownWidth, focalLength, perWidth):
    # вычислить и вернуть расстояние от объекта до камеры
    return (knownWidth * focalLength) / perWidth

In [3]:
# инициализируем известное расстояние от камеры до объекта, которое в данном случае 15.748 дюйма
KNOWN_DISTANCE = 15.748
# инициализируем известную ширину объекта, в данном случае лист бумаги шириной 11.7 дюймов
KNOWN_WIDTH = 11.7

# загружаем первое изображение, содержащее объект, который находится в 15.748 дюймах(40см) от нашей камеры,
# затем находим лист А4 (маркер) на изображении и инициализируем фокусное расстояние
image = cv2.imread("40.jpg")
marker = find_marker(image)
focalLength = (marker[1][0] * KNOWN_DISTANCE) / KNOWN_WIDTH

In [10]:
# загружаем изображение, находим маркер на изображении, затем вычисляем расстояние от камеры до маркера
image1 = cv2.imread("40.jpg")
image = cv2.resize(image1, (800, 600))
marker = find_marker(image)
cm = distance_to_camera(KNOWN_WIDTH, focalLength, marker[1][0])
# рисуем ограничивающую рамку вокруг маркера и отображаем ее
box = cv2.cv.BoxPoints(marker) if imutils.is_cv2() else cv2.boxPoints(marker)
box = np.int0(box)
cv2.drawContours(image, [box], -1, (0, 255, 0), 2)
cv2.putText(image, "%.2fm" % (cm / 100),
        (image.shape[1] - 200, image.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,
        2.0, (0, 255, 0), 3)

cv2.imshow("image", image)
cv2.waitKey(0)

-1